In [1]:
print(1)

1


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2))
        x = x.view(1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        
        return num_features
        

In [12]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [17]:
input = Variable(torch.randn(1,1, 32,32))

In [18]:
input

tensor([[[[ 1.2819, -1.9193,  0.0955,  ...,  1.2230,  1.1611, -1.6812],
          [-1.1083, -1.3858, -0.2273,  ..., -0.1673, -0.7143,  0.7890],
          [-1.0321, -0.8284,  0.3813,  ..., -1.4754,  1.2908,  0.2549],
          ...,
          [-1.5174,  2.3560,  1.2034,  ..., -0.0194, -1.2406,  0.2900],
          [ 1.2955,  0.3333, -0.9296,  ..., -0.0190, -0.5210,  1.9129],
          [-1.4030,  0.3824,  0.3754,  ...,  0.2172,  1.0992,  1.2143]]]])

In [21]:
output = net(input)

In [16]:
16 * 13 * 13

2704

In [22]:
net.zero_grad()

In [24]:
output.backward(torch.randn(1, 10))

In [26]:
output= net(input)

In [27]:
target = torch.randn(1, 10)

In [28]:
target

tensor([[-0.2686,  0.4977,  0.4980,  1.3779, -0.2374, -1.0782, -0.7577,  0.7264,
          1.0674,  0.2750]])

In [29]:
target = target.view(1, -1)  # make it the same shape as output


In [31]:
criterion = nn.MSELoss()

loss = criterion(output, target)

In [32]:
loss

tensor(0.5746, grad_fn=<MseLossBackward0>)

In [35]:
loss.grad_fn

In [37]:
loss.grad_fn.next_functions[0][0]

In [38]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]

In [39]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0048,  0.0108,  0.0042,  0.0087,  0.0045, -0.0018])
